# ReverseNumber Example - two users

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os, json

from colink import CoLink

from aiflows.utils import serve_utils
from aiflows.utils.general_helpers import read_yaml_file
from aiflows.messages import FlowMessage
from aiflows.utils import coflows_utils, colink_utils
from aiflows.workers import run_dispatch_worker_thread, run_mount_worker_thread

def status(cl):
    colink_utils.print_served_flows(cl, print_values=False)
    print("\nAll flow instances:")
    colink_utils.print_flow_instances(cl)

# Create two users

In [3]:
cls = colink_utils.start_colink_server_with_users(num_users=2)

# USER 0

In [4]:
run_dispatch_worker_thread(cls[0])
run_mount_worker_thread(cls[0])

Dispatch worker started in attached thread.
dispatch_point: coflows_dispatch
Mount worker started in attached thread for user  0392babafe816fc3f09483292e09359cffea9e7a08236469a33d7a759640e9b254

~~~ Mount Receiver Task ~~~
Mounted d5117c38-7fc2-4551-b754-6316cff9279c at flows:reverse_number_atomic:mounts:03ee4d7636622d2de06b7f1ecb2f6abc46357c0de279a33c90bf6d0f1c6f000c1f:d5117c38-7fc2-4551-b754-6316cff9279c
Fetched singleton d5117c38-7fc2-4551-b754-6316cff9279c

~~~ Dispatch task ~~~
flow_type: reverse_number_atomic
flow_id: d5117c38-7fc2-4551-b754-6316cff9279c
message_paths: ['push_tasks:6baaff05-4cee-4d0d-9aa7-da417412179c:msg']
parallel_dispatch: False

Called ReverseNumberAtomic 
 state {}

~~~ Dispatch task ~~~
flow_type: reverse_number_atomic
flow_id: d5117c38-7fc2-4551-b754-6316cff9279c
message_paths: ['push_tasks:e81b34ea-a7b2-48fd-910d-23b23b405db4:msg']
parallel_dispatch: False

Called ReverseNumberAtomic 
 state {}


In [5]:
serve_utils.serve_flow(
    cl=cls[0],
    flow_class_name="ReverseNumberFlowModule.ReverseNumberAtomicFlow",
    flow_endpoint="reverse_number_atomic",
    singleton=True,
)

Started serving ReverseNumberFlowModule.ReverseNumberAtomicFlow at flows:reverse_number_atomic.
dispatch_point: coflows_dispatch
parallel_dispatch: False
singleton: True


True

# USER 1

In [6]:
run_dispatch_worker_thread(cls[1])
run_mount_worker_thread(cls[1])

Dispatch worker started in attached thread.
dispatch_point: coflows_dispatch
Mount worker started in attached thread for user  03ee4d7636622d2de06b7f1ecb2f6abc46357c0de279a33c90bf6d0f1c6f000c1f

~~~ Mount Initiator Task ~~~
Received subflow ids: {
    "first_reverse_flow": "d5117c38-7fc2-4551-b754-6316cff9279c",
    "second_reverse_flow": "d5117c38-7fc2-4551-b754-6316cff9279c"
}

~~~ Dispatch task ~~~
flow_type: reverse_number_sequential
flow_id: e1a75fe3-4c0f-4233-bbef-1a2c0ed7b7eb
message_paths: ['push_tasks:5928c193-e82e-4484-9144-61407c874683:msg']
parallel_dispatch: False

Called ReverseNumberSequential 
 state {'current_call': 'first_reverse_flow'}

~~~ Dispatch task ~~~
flow_type: reverse_number_sequential
flow_id: e1a75fe3-4c0f-4233-bbef-1a2c0ed7b7eb
message_paths: ['push_tasks:5200fc0d-5620-45e2-9208-455366ea8b0a:msg']
parallel_dispatch: False

Called ReverseNumberSequential 
 state {'current_call': 'second_reverse_flow', 'initial_message': FlowMessage(message_id='9e0fdfb0-116

In [7]:
serve_utils.serve_flow(
    cl=cls[1],
    flow_class_name="ReverseNumberFlowModule.ReverseNumberSequentialFlow",
    flow_endpoint="reverse_number_sequential",
)

Started serving ReverseNumberFlowModule.ReverseNumberSequentialFlow at flows:reverse_number_sequential.
dispatch_point: coflows_dispatch
parallel_dispatch: False
singleton: False


True

In [8]:
# since we are creating new dummy users on every run, we inject user_id into config manually
# typically this would be hardcoded in yaml config
cfg_overrides = {
    "subflows_config": {
        "first_reverse_flow": {
            "user_id": cls[0].get_user_id()
        },
        "second_reverse_flow": {
            "user_id": cls[0].get_user_id()
        }
    }
}

flow = serve_utils.get_flow_instance(
    cl=cls[1],
    flow_endpoint="reverse_number_sequential",
    user_id="local",
    config_overrides = cfg_overrides,
)

Mounted e1a75fe3-4c0f-4233-bbef-1a2c0ed7b7eb at flows:reverse_number_sequential:mounts:local:e1a75fe3-4c0f-4233-bbef-1a2c0ed7b7eb


In [9]:
input_data = {"id": 0, "number": 1234}
    
input_message = flow.package_input_message(input_data)
future = flow.get_reply_future(input_message)
reply_data = future.get_data()

print("Data sent:\n",  input_data, "\n")
print("REPLY:\n", reply_data, "\n")

Data sent:
 {'id': 0, 'number': 1234} 

REPLY:
 {'output_number': 1234} 



# Observe storage of both users

User 0 has a serve endpoint for ReverseNumberAtomic flow and a singleton instance of that flow.
User 1 has a serve endpoint for ReverseNumberSequential flow and an instance of that flow whose subflows are remote instances of the ReverseNumberAtomic at user 0.

In [10]:
status(cls[0])

 reverse_number_atomic/
   flow_class_name
   init
   mounts/
     03ee4d7636622d2de06b7f1ecb2f6abc46357c0de279a33c90bf6d0f1c6f000c1f/
       d5117c38-7fc2-4551-b754-6316cff9279c/
         config_overrides
         state
         init
   singleton
   parallel_dispatch
   default_dispatch_point

All flow instances:
 d5117c38-7fc2-4551-b754-6316cff9279c


In [11]:
status(cls[1])

 reverse_number_sequential/
   singleton
   flow_class_name
   default_dispatch_point
   init
   parallel_dispatch
   mounts/
     local/
       e1a75fe3-4c0f-4233-bbef-1a2c0ed7b7eb/
         state
         init
         config_overrides

All flow instances:
 e1a75fe3-4c0f-4233-bbef-1a2c0ed7b7eb
